In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cd

/root


In [8]:
cd /content/drive/My Drive

/content/drive/My Drive


In [0]:
tf.__version__

'1.13.1'

In [0]:
%%capture
!pip install watermark


In [0]:
%load_ext watermark
%watermark -p tensorflow,numpy -m

In [0]:
pip install tensorflow==1.13.1

     |████████████████████████████████| 92.5MB 1.2MB/s 
     |████████████████████████████████| 3.2MB 39.5MB/s 
     |████████████████████████████████| 368kB 40.6MB/s 
  Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0
  Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
      Successfully uninstalled tensorflow-estimator-1.14.0
  Found existing installation: tensorflow 1.14.0
    Uninstalling tensorflow-1.14.0:
      Successfully uninstalled tensorflow-1.14.0


In [0]:
from keras.datasets import mnist
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint,CSVLogger
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Conv2D,MaxPooling2D,AveragePooling2D,Flatten,UpSampling2D,Conv2DTranspose,Input,BatchNormalization,LeakyReLU
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
import natsort
from keras.utils import multi_gpu_model

In [15]:
from keras.preprocessing.image import ImageDataGenerator
datagen=ImageDataGenerator(rescale=1.0/255.0,validation_split=0.1)
datagen1=ImageDataGenerator(rescale=1.0/255.0,validation_split=0.1)
datagen2=ImageDataGenerator(rescale=1.0/255.0,validation_split=0.1)
datagen3=ImageDataGenerator(rescale=1.0/255.0,validation_split=0.1)

seed=1
t_g=datagen.flow_from_directory('/content/drive/My Drive/dataset/H/ITSI',target_size=(512,512),class_mode=None,classes=['haze'],batch_size=1,seed=seed)
d_g=datagen1.flow_from_directory('/content/drive/My Drive/dataset/H/ITSO',target_size=(512,512),class_mode=None,classes=['dehaze'],batch_size=1,seed=seed)
v_g=datagen2.flow_from_directory('/content/drive/My Drive/SOTS/indoor/HaZy',target_size=(512,512),class_mode=None,classes=['hazy'],batch_size=1,seed=seed)
c_g=datagen2.flow_from_directory('/content/drive/My Drive/SOTS/indoor/GT_i',target_size=(512,512),class_mode=None,classes=['gt'],batch_size=1,seed=seed)

train_gen=zip(t_g,d_g)
test_gen=zip(v_g,c_g)
print(train_gen)
#print(t_g[1].shape)
def combine_generator(gen1, gen2):
  while True:
    yield(gen1.next(), gen2.next())

combinedGenerator = combine_generator(t_g,d_g)
combinedGenerator1 = combine_generator(v_g,c_g)

Found 13920 images belonging to 1 classes.
Found 13920 images belonging to 1 classes.
Found 500 images belonging to 1 classes.
Found 500 images belonging to 1 classes.


#gang

In [0]:
!pip install -q tensorflow==1.13.1

Using TensorFlow backend.


In [0]:
print(len(os.listdir("/content/drive/My Drive/mixi/haze")))
print(len(os.listdir("/content/drive/My Drive/mixo/dehaze")))

In [0]:
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']

Found 0 images belonging to 1 classes.
Found 9171 images belonging to 1 classes.
Found 492 images belonging to 1 classes.
Found 492 images belonging to 1 classes.


In [0]:
import os
try:
    device_name = os.environ['COLAB_TPU_ADDR']
    TPU_ADDRESS = 'grpc://' + device_name
    print('Found TPU at: {}'.format(TPU_ADDRESS))
except KeyError:
    print('TPU not found')

Found TPU at: grpc://10.68.46.130:8470


In [0]:

def mod():
  inputs = Input(shape=(512, 512, 3))

  x=Conv2D(50,(3,3),activation='relu',strides=1,padding='same')(inputs)
  x=AveragePooling2D((2,2))(x)
  x=Conv2D(50,(3,3),activation='relu',strides=1,padding='same')(x) 
  x=AveragePooling2D((2,2))(x)
  x=Dense(10,activation='relu')(x)
  x=Dense(10,activation='relu')(x)


  x=Conv2DTranspose(50,(3,3),activation='relu',strides=1,padding='same')(x)
  x=UpSampling2D((2,2))(x)
  x=Conv2DTranspose(50,(3,3),activation='relu',strides=1,padding='same')(x)
  x=UpSampling2D((2,2))(x)



  x=Conv2DTranspose(3,(3,3),activation='linear',strides=1,padding='same')(x)
  auto=Model(inputs,x)
  auto.compile
  return Model(inputs, x)


autoencoder = mod()
autoencoder.compile(loss='mean_squared_error',optimizer='adam',metrics=['accuracy'])
#print(autoencoder.summary())


#autoencoder.fit_generator(combinedGenerator,epochs=10,verbose=1,steps_per_epoch=13920,validation_data=combinedGenerator1,validation_steps=500)
#prediction=autoencoder.predict(X)
#yo=autoencoder.predict(X1)
#plt.imshow(prediction[0])
#plt.show()


In [0]:
autoencoder=tf.keras.models.load_model("9171model3.model")

W0718 04:25:28.217699 139708964870016 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0718 04:25:28.219331 139708964870016 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0718 04:25:28.223101 139708964870016 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtyp

In [0]:
p=autoencoder.predict()

In [19]:
filepath="/content/drive/My Drive/chkpoints/1-{epoch:02d}-{val_acc:.2f}-{acc:.3f}.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
# = EarlyStopping(monitor='val_loss', patience=15, verbose=1)
csv_logger = CSVLogger('/content/drive/My Drive/chkpoints/training.csv',separator=',', append=True)
callbacks_list = [checkpoint,csv_logger]
autoencoder.fit_generator(combinedGenerator,epochs=10,verbose=1,steps_per_epoch=13920,callbacks=callbacks_list,validation_data=combinedGenerator1,validation_steps=500)


Epoch 1/10
13919/13920 [============================>.] - ETA: 0s - loss: 0.0771 - acc: 0.7823
Epoch 00001: val_acc improved from -inf to 0.83995, saving model to /content/drive/My Drive/chkpoints/1-01-0.84-0.782.hdf5
13920/13920 [==============================] - 5533s 397ms/step - loss: 0.0771 - acc: 0.7823 - val_loss: 0.0848 - val_acc: 0.8400
Epoch 2/10
13918/13920 [============================>.] - ETA: 0s - loss: 0.0763 - acc: 0.7811
Epoch 00002: val_acc did not improve from 0.83995
13920/13920 [==============================] - 691s 50ms/step - loss: 0.0763 - acc: 0.7811 - val_loss: 0.0690 - val_acc: 0.8396
Epoch 3/10
13919/13920 [============================>.] - ETA: 0s - loss: 0.0754 - acc: 0.7810
Epoch 00003: val_acc improved from 0.83995 to 0.84062, saving model to /content/drive/My Drive/chkpoints/1-03-0.84-0.781.hdf5
13920/13920 [==============================] - 709s 51ms/step - loss: 0.0754 - acc: 0.7810 - val_loss: 0.0726 - val_acc: 0.8406
Epoch 4/10
13918/13920 [======

In [0]:
autoencoder.save('itsmomdel.model')

In [0]:
resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)
tf.contrib.distribute.initialize_tpu_system(resolver)
strategy = tf.contrib.distribute.TPUStrategy(resolver)

In [0]:
strategy = tf.distribute.experimental.TPUStrategy(tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS))
with strategy.scope():
  autoencoder = mod()
  autoencoder= model_to_tpu(autoencoder)
  autoencoder.compile( loss='mean_squared_error',optimizer='adam',metrics=['accuracy'])
  autoencoder.fit_generator(combinedGenerator,epochs=40,verbose=1,steps_per_epoch=9171,validation_data=combinedGenerator1,validation_steps=492)
"""with strategy.scope():
  autoencoder = mod()
  autoencoder= model_to_tpu(autoencoder)
  autoencoder.compile( loss='mean_squared_error',optimizer='adam',metrics=['accuracy'])
  
autoencoder.fit_generator(combinedGenerator,epochs=40,verbose=1,steps_per_epoch=9171,validation_data=combinedGenerator1,validation_steps=492)"""

In [0]:
tpu_model1 = tf.contrib.tpu.keras_to_tpu_model(
    autoencoder,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
    tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS))
  )

tpu_model1.compile(
     loss='mean_squared_error',optimizer='adam',metrics=['accuracy']
  )


INFO:tensorflow:Querying Tensorflow master (grpc://10.68.46.130:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 2162416078239639620)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 14905014219472558206)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 15158806226609898598)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 4310284092217973500)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 7966899917571117636)
INFO:tensorflow:*** Available Device: _DeviceAttributes(

In [0]:
tpu_model1.fit_generator(combinedGenerator,epochs=40,verbose=1,steps_per_epoch=9171,validation_data=combinedGenerator1,validation_steps=492)


Epoch 1/40
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(0,), dtype=tf.int32, name='core_id_20'), TensorSpec(shape=(0, 512, 512, 3), dtype=tf.float32, name='input_1_20'), TensorSpec(shape=(0, 512, 512, 3), dtype=tf.float32, name='conv2d_transpose_2_target_30')]


ValueError: ignored

In [0]:
resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)
tf.contrib.distribute.initialize_tpu_system(resolver)
strategy = tf.contrib.distribute.TPUStrategy(resolver)

with strategy.scope():
  autoencoder = mod()
  autoencoder= model_to_tpu(autoencoder)
  autoencoder.compile( loss='mean_squared_error',optimizer='adam',metrics=['accuracy'])
  
autoencoder.fit_generator(combinedGenerator,epochs=40,verbose=1,steps_per_epoch=9171,validation_data=combinedGenerator1,validation_steps=492)



#parallel_model = multi_gpu_model(autoencoder, gpus=8)
#parallel_model.compile(optimizer="adam",loss="mse",metrics=["accuracy"])
#parallel_model.fit_generator(combinedGenerator,epochs=10,verbose=1,steps_per_epoch=9171,validation_data=combinedGenerator1,validation_steps=492,use_multiprocessing=True,workers=16)


In [0]:
def model_to_tpu(model):
    return tf.contrib.tpu.keras_to_tpu_model( model, strategy=tf.contrib.tpu.TPUDistributionStrategy(tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)))

In [0]:
autoencoder=mod()
#autoencoder.compile( loss='mean_squared_error',optimizer='adam',metrics=['accuracy'])
tpu_model= model_to_tpu(autoencoder)

#start


In [0]:
import os
import pprint
import tensorflow as tf

if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.compat.v1.Session(tpu_address) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  pprint.pprint(devices)

TPU address is grpc://10.1.144.34:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 11576568105333922131),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 17598324169068066291),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 4139609080810708531),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 3618200149069864916),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 14315048814185123357),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 7398793154207577700),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 8755232771065159063),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 12771375440801377607),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 1701500605630

In [0]:
inputs = Input(shape=(512, 512, 3))
x=Conv2D(50,(3,3),activation='relu',strides=1,padding='same')(inputs)
x=AveragePooling2D((2,2))(x)
x=Conv2D(50,(3,3),activation='relu',strides=1,padding='same')(x) 
x=AveragePooling2D((2,2))(x)
x=Dense(10,activation='relu')(x)
x=Dense(10,activation='relu')(x)


x=Conv2DTranspose(50,(3,3),activation='relu',strides=1,padding='same')(x)
x=UpSampling2D((2,2))(x)
x=Conv2DTranspose(50,(3,3),activation='relu',strides=1,padding='same')(x)
x=UpSampling2D((2,2))(x)
x=Conv2DTranspose(3,(3,3),activation='linear',strides=1,padding='same')(x)
auto=Model(inputs,x)
  

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    auto,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    )
)
tpu_model.compile(
     loss='mean_squared_error',optimizer=tf.train.RMSPropOptimizer(learning_rate=0.001),metrics=['accuracy']
)

INFO:tensorflow:Querying Tensorflow master (grpc://10.1.144.34:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 11576568105333922131)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 17598324169068066291)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 4139609080810708531)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 3618200149069864916)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 14315048814185123357)
INFO:tensorflow:*** Available Device: _DeviceAttributes(

In [0]:
tpu_model.fit_generator(
    combinedGenerator,epochs=40,verbose=1,steps_per_epoch=9171,validation_data=combinedGenerator1,validation_steps=492
)

Epoch 1/40
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(1,), dtype=tf.int32, name='core_id_60'), TensorSpec(shape=(1, 512, 512, 3), dtype=tf.float32, name='input_1_60'), TensorSpec(shape=(1, 512, 512, 3), dtype=tf.float32, name='conv2d_transpose_2_target_70')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input_1
INFO:tensorflow:Started compiling


RuntimeError: ignored